In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision
import os


transform = transforms.Compose([#transforms.Resize((256,256)),  
                                transforms.Grayscale(),		# the code transforms.Graysclae() is for changing the size [3,100,100] to [1, 100, 100] (notice : [channel, height, width] )
                                transforms.ToTensor(),])


#train_data_path = 'relative path of training data set'
train_data_path = 'horse-or-human/horse-or-human/train'
trainset = torchvision.datasets.ImageFolder(root=train_data_path, transform=transform)
# change the valuse of batch_size, num_workers for your program
# if shuffle=True, the data reshuffled at every epoch #numworks가 뭐지?
trainloader = torch.utils.data.DataLoader(trainset, batch_size=1027, shuffle=False, num_workers=0)  


validation_data_path = 'horse-or-human/horse-or-human/validation'
valset = torchvision.datasets.ImageFolder(root=validation_data_path, transform=transform)
# change the valuse of batch_size, num_workers for your program
valloader = torch.utils.data.DataLoader(valset, batch_size=256, shuffle=False, num_workers=0)  

NUM_EPOCH=1
for epoch in range(NUM_EPOCH):
    # load training images of the batch size for every iteration
    for i, data in enumerate(trainloader):

        # inputs is the image
        # labels is the class of the image
        inputs, labels = data

        # if you don't change the image size, it will be [batch_size, 1, 100, 100]
        #print(inputs.shape)

        # if labels is horse it returns tensor[0,0,0] else it returns tensor[1,1,1]
        #print(labels)  





    # load validation images of the batch size for every iteration
    for i, data in enumerate(valloader):
        
        # inputs is the image
        # labels is the class of the image
        inputs, labels = data

        # if you don't change the image size, it will be [batch_size, 1, 100, 100]
        #print(inputs.shape)

        # if labels is horse it returns tensor[0,0,0] else it returns tensor[1,1,1]
        #print(labels)    

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import torch.nn as nn
import numpy as np
import plotly.graph_objects as go

In [3]:
train_dataset = next(iter(trainloader))[0].numpy()
train_label =next(iter(trainloader))[1].numpy()

val_dataset = next(iter(valloader))[0].numpy()
val_label =next(iter(valloader))[1].numpy()

In [4]:
train_label=train_label.reshape(1027,1)
val_label=val_label.reshape(256,1)

In [5]:
train_flatten=train_dataset.reshape(1027,train_dataset.shape[2]*train_dataset.shape[3])
val_flatten=val_dataset.reshape(256,val_dataset.shape[2]*val_dataset.shape[3])

In [6]:
x_train=train_flatten.T
y_train=train_label.T
print(x_train.shape)
print(y_train.shape)

x_test=val_flatten.T
y_test=val_label.T
print(x_test.shape)
print(y_test.shape)


(10000, 1027)
(1, 1027)
(10000, 256)
(1, 256)


In [7]:
def initialize_weights_and_bias(dimension):
    w = np.full((dimension,1),0.01)
    b = 0.0
    return w, b

def sigmoid(z):
    y_head = 1/(1+np.exp(-z))
    return y_head

def forward_backward_propagation(w,b,x_train,y_train):
    delta=1e-7
    # forward propagation
    z = np.dot(w.T,x_train) + b
    y_head = sigmoid(z)
    loss = -y_train*np.log(y_head+delta)-(1-y_train)*np.log((1-y_head)+delta)
    cost = (np.sum(loss))/x_train.shape[1]
    # backward propagation
    derivative_weight = (np.dot(x_train,((y_head-y_train).T)))/x_train.shape[1]
    derivative_bias = np.sum(y_head-y_train)/x_train.shape[1]
    gradients = {"derivative_weight": derivative_weight,"derivative_bias": derivative_bias}
    return cost,gradients

def loss_fnc(w,b,x_test,y_test):
    delta=1e-7
    z = np.dot(w.T,x_test) + b
    y_head = sigmoid(z)
    loss = -y_test*np.log(y_head+delta)-(1-y_test)*np.log((1-y_head)+delta)
    cost = (np.sum(loss))/x_test.shape[1]
    return cost

def update(w, b, x_train, y_train, learning_rate,number_of_iterarion):
    cost_list = []
    cost_list2 = []
    index = []
    val_list = []
    ac_train=[]
    ac_val=[]
    
    for i in range(number_of_iterarion):
        
        cost,gradients = forward_backward_propagation(w,b,x_train,y_train)
        cost_list.append(cost)
        
        w = w - learning_rate * gradients["derivative_weight"]
        b = b - learning_rate * gradients["derivative_bias"]
        if i % 100 == 0:
            cost_list2.append(cost)
            index.append(i)
            val_list.append(loss_fnc(w,b,x_test,y_test))
           # print ("Cost after iteration %i: %f" %(i, cost))
            print("Iteration %i\t Training Loss : %f\t Validation Loss %f"%(i,cost,loss_fnc(w,b,x_test,y_test)))
            parameters = {"weight": w,"bias": b}
            y_prediction_test = predict(parameters["weight"],parameters["bias"],x_test)
            y_prediction_train = predict(parameters["weight"],parameters["bias"],x_train)
            print("\t\t Train Accuracy: {} \t\t Test Accuracy: {} ".format(round(100 - np.mean(np.abs(y_prediction_train - y_train)) * 100,2),round(100 - np.mean(np.abs(y_prediction_test - y_test)) * 100,2)))
            ac_train.append((round(100 - np.mean(np.abs(y_prediction_train - y_train)) * 100,2)))
            ac_val.append(round(100 - np.mean(np.abs(y_prediction_test - y_test)) * 100,2))
    
                            
    parameters = {"weight": w,"bias": b}
    plt.plot(index,cost_list2)
    plt.xticks(index,rotation='vertical')
    plt.xlabel("Number of Iterations")
    plt.ylabel("Loss of Training Set")
    plt.show()
    
    plt.plot(index,val_list)
    plt.xticks(index,rotation='vertical')
    plt.xlabel("Number of Iterations")
    plt.ylabel("Loss of Validation Set")
    plt.show()
                          
    plt.plot(index,ac_train)
    plt.xticks(index,rotation='vertical')
    plt.xlabel("Number of Iterations")
    plt.ylabel("Accuracy of Training Set")
    plt.show()
                            
    plt.plot(index,ac_val)
    plt.xticks(index,rotation='vertical')
    plt.xlabel("Number of Iterations")
    plt.ylabel("Accuracy of Validation Set")
    plt.show()
                            
    return parameters, gradients, cost_list, val_list

def predict(w,b,x_test):
    
    z = sigmoid(np.dot(w.T,x_test)+b)
    Y_prediction = np.zeros((1,x_test.shape[1]))

    for i in range(z.shape[1]):
        if z[0,i]<= 0.5:
            Y_prediction[0,i] = 0
        else:
            Y_prediction[0,i] = 1

    return Y_prediction

def logistic_regression(x_train, y_train, x_test, y_test, learning_rate ,  num_iterations):

    dimension =  x_train.shape[0]
    w,b = initialize_weights_and_bias(dimension)

    parameters, gradients, cost_list, val_list = update(w, b, x_train, y_train, learning_rate,num_iterations)
    
    y_prediction_test = predict(parameters["weight"],parameters["bias"],x_test)
    y_prediction_train = predict(parameters["weight"],parameters["bias"],x_train)
    
    print("Final Results :")
    #print("\tTrain Accuracy: {} %".format(round(100 - np.mean(np.abs(y_prediction_train - y_train)) * 100,2)))
    #print("\tTrain Loss: %.4f " %(cost_list[-1]))
    #print("\tValidation Accuracy: {}%".format(round(100 - np.mean(np.abs(y_prediction_test - y_test)) * 100,2)))
    #print("\tValidation Loss: %.4f"%(val_list[-1]))
    
    

    headerColor = 'grey'
    rowEvenColor = 'lightgrey'
    rowOddColor = 'white'

    fig = go.Figure(data=[go.Table(
    header=dict(
    values=['<b>dataset</b>','<b>loss</b>','<b>accuracy</b>'],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)),
    cells=dict(
    values=[
      ['training', 'validation'],
      ["{0:.6f}".format(cost_list[-1]), "{0:.6f}".format(val_list[-1])],
      [(round(100 - np.mean(np.abs(y_prediction_train - y_train)) * 100,2)), (round(100 - np.mean(np.abs(y_prediction_test - y_test)) * 100,2))]],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))])

    fig.show()

In [ ]:
logistic_regression(x_train, y_train, x_test, y_test,learning_rate = 0.01, num_iterations = 2000)

Iteration 0	 Training Loss : 7.845892	 Validation Loss 8.059048
		 Train Accuracy: 51.31 		 Test Accuracy: 50.0 
Iteration 100	 Training Loss : 1.699862	 Validation Loss 1.958832
		 Train Accuracy: 58.33 		 Test Accuracy: 57.42 
Iteration 200	 Training Loss : 0.925984	 Validation Loss 1.178888
		 Train Accuracy: 67.87 		 Test Accuracy: 64.84 
Iteration 300	 Training Loss : 0.703938	 Validation Loss 1.043585
		 Train Accuracy: 74.2 		 Test Accuracy: 69.14 
Iteration 400	 Training Loss : 0.575685	 Validation Loss 0.986026
		 Train Accuracy: 78.97 		 Test Accuracy: 70.31 
Iteration 500	 Training Loss : 0.508078	 Validation Loss 0.970229
		 Train Accuracy: 80.62 		 Test Accuracy: 70.7 
Iteration 600	 Training Loss : 0.260244	 Validation Loss 0.477116
		 Train Accuracy: 88.7 		 Test Accuracy: 82.03 
Iteration 700	 Training Loss : 0.246325	 Validation Loss 0.520084
		 Train Accuracy: 89.29 		 Test Accuracy: 80.47 
Iteration 800	 Training Loss : 0.254306	 Validation Loss 0.623490
		 Train Acc